In [ ]:
from gitloc import get_commits
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.dpi'] = 300

commits = get_commits()
commits = get_commits("../my-favorite-repo")

df = pd.DataFrame(commits).set_index('date')
df["net"] = df["adds"] - df["deletes"]
df["locs"] = df["net"].cumsum()
sns.scatterplot(df["net"], s=2)
sns.lineplot(df["locs"])
